In [1]:
import pandas as pd
from fastparquet import ParquetFile
import duckdb
import requests
import re

In [2]:
stkr=  ParquetFile("sticker.parquet").to_pandas()

In [70]:
autographs = duckdb.query("select * from stkr where type = 'Autograph'").df()
autographs.head()

,date,price,volume,item,type,effect,market_hash_name,rarity,tournament_id,tournament,team_id,team_tag,team,image_url,rarity_color
0,2019-08-14 01:00:00,0.818,35,Sticker | Aleksib (Foil) | Berlin 2019,Autograph,Foil,Sticker | Aleksib (Foil) | Berlin 2019,Remarkable,16.0,2019 StarLadder Berlin,84.0,ENCE,ENCE,https://community.akamai.steamstatic.com/econo...,purple
1,2019-08-15 01:00:00,0.745,127,Sticker | Aleksib (Foil) | Berlin 2019,Autograph,Foil,Sticker | Aleksib (Foil) | Berlin 2019,Remarkable,16.0,2019 StarLadder Berlin,84.0,ENCE,ENCE,https://community.akamai.steamstatic.com/econo...,purple
2,2019-08-16 01:00:00,0.789,54,Sticker | Aleksib (Foil) | Berlin 2019,Autograph,Foil,Sticker | Aleksib (Foil) | Berlin 2019,Remarkable,16.0,2019 StarLadder Berlin,84.0,ENCE,ENCE,https://community.akamai.steamstatic.com/econo...,purple
3,2019-08-17 01:00:00,0.893,57,Sticker | Aleksib (Foil) | Berlin 2019,Autograph,Foil,Sticker | Aleksib (Foil) | Berlin 2019,Remarkable,16.0,2019 StarLadder Berlin,84.0,ENCE,ENCE,https://community.akamai.steamstatic.com/econo...,purple
4,2019-08-18 01:00:00,0.826,55,Sticker | Aleksib (Foil) | Berlin 2019,Autograph,Foil,Sticker | Aleksib (Foil) | Berlin 2019,Remarkable,16.0,2019 StarLadder Berlin,84.0,ENCE,ENCE,https://community.akamai.steamstatic.com/econo...,purple


In [71]:
names = duckdb.query("select distinct item from autographs where type = 'Autograph'").df()

In [77]:
parsed = {name[0]: re.match(r'Sticker \| (([^\(|\|])+)',name[0]).group(1).strip() for name in names.values.tolist()}

mappings = pd.Series(parsed).to_frame().reset_index()

mappings.columns = ["sticker_name", 'player']

duckdb.query("select * from autographs join mappings on autographs.item = mappings.sticker_name").df().drop(
    columns=['type', 'market_hash_name', 'image_url', 'sticker_name'], axis=1
).to_parquet('autographs.parquet')